In [1]:
import os
import requests
from typing import List
import json
from bs4 import BeautifulSoup
from IPython.display import HTML, Markdown, display, update_display
import google.generativeai as genai  
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv


load_dotenv(find_dotenv())
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


model = genai.GenerativeModel(model_name="gemini-1.5-flash")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class Website:
    url : str
    title : str
    body : str
    text : str
    links : list[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else " No Title Found"

        if soup.body:
            for irrelvant_tag in soup.body(['script', 'style', 'header', 'footer', 'nav']):
                irrelvant_tag.decompose()

            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.title = ""

        

        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link ]  

    def get_contents(self):
        return f"""
                Title: {self.title}
                Text: {self.text}
                Links: {self.links}
            """
ed = Website("https://www.infosys.com/")
#print(ed.get_content())

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

#print(link_system_prompt)


def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links_response(url):
    website = Website(url)
    user_prompt = get_links_user_prompt(website)
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

def get_all_details(url):
    result = "Landing Page Details:\n"
    links = get_links_response(url)
    print("Found links:", links)
    for link in links["links"]:
        print(f"Link: {link['url']} - {link['type']}")
        website = Website(link["url"])
        result += f"Content: {website.get_contents()}\n\n"
    return result

print(get_all_details("https://www.infosys.com/"))

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, website_url):
    user_prompt = f"Company Name: {company_name}\n"     
    user_prompt += f"Website URL: {website_url}\n"
    user_prompt += "Please create a brochure about the company for prospective customers, investors and recruits. \
Include details of company culture, customers and careers/jobs if you have the information."
    return user_prompt

def create_brochure(company_name, url):
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))








#response = client.models.generate_content(
#    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
#)
#print(response.text)

c:\Kichu AI\LLMCode\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found links: {'links': [{'type': 'about page', 'url': 'https://www.infosys.com/about.html'}, {'type': 'careers page', 'url': 'https://www.infosys.com/careers.html'}, {'type': 'services page', 'url': 'https://www.infosys.com/services.html'}, {'type': 'industries page', 'url': 'https://www.infosys.com/industries.html'}, {'type': 'products and platforms page', 'url': 'https://www.infosys.com/products-and-platforms.html'}, {'type': 'investors page', 'url': 'https://www.infosys.com/investors.html'}]}
Link: https://www.infosys.com/about.html - about page
Link: https://www.infosys.com/careers.html - careers page
Link: https://www.infosys.com/services.html - services page
Link: https://www.infosys.com/industries.html - industries page
Link: https://www.infosys.com/products-and-platforms.html - products and platforms page
Link: https://www.infosys.com/investors.html - investors page
Landing Page Details:
Content: 
                Title: Infosys - Global Leader in Technology Services and Consult

In [2]:
create_brochure("Infosys", "https://www.infosys.com")

# Infosys Brochure

---

## About Us

**Company Name:** Infosys  
**Website:** [Infosys](https://www.infosys.com)

Infosys is a global leader in technology services and consulting. Operating from over 46 countries, we empower businesses to adapt and thrive in the digital era. With a focus on innovation, we utilize advanced technologies to help our clients achieve their goals and elevate their operational efficiency.

---

## Our Mission

To amplify human potential and create a more prosperous world through technology and innovation.

---

## Company Culture

At Infosys, we embrace a culture of inclusivity and diversity. Our values are rooted in integrity, collaboration, and customer-centricity. We believe in fostering an environment where creativity and innovation flourish. Our workplace promotes:

- **Learning**: Continuous growth through training and development programs.
- **Collaboration**: Encouraging teamwork across various domains and geographies.
- **Innovation**: Emphasizing creative thinking to solve real-world challenges.
- **Sustainability**: Committed to environmental and social responsibility.

Our employees are our greatest asset, and we strive to provide them with a work environment that inspires, supports, and enables them to excel.

---

## Our Customers

Infosys serves a wide range of industries, including:

- **Financial Services**
- **Healthcare**
- **Manufacturing**
- **Retail**
- **Telecommunications**

We work with clients ranging from start-ups to Fortune 500 companies, helping them harness cutting-edge technology to drive digital transformation, improve operational efficiency and create value for their stakeholders.

---

## Careers at Infosys

Our dynamic teams are always on the lookout for talented individuals who are eager to drive change and develop game-changing solutions. At Infosys, you can expect:

- **Innovative Work Environment**: Engage in projects that leverage advanced technologies like AI, IoT, and cloud computing.
- **Career Growth**: Clear pathways for advancement, mentoring programs, and continuous learning opportunities.
- **Work-Life Balance**: Flexible work options and support for personal wellbeing.
- **Global Exposure**: Opportunities to work with diverse teams across different cultures and regions.

**Join us** [here](https://www.infosys.com/careers/) to explore current job openings and submit your application. Be a part of a team that’s redefining the future of technology!

---

## Connect With Us

For more information about our services, solutions, and career opportunities, visit our website at [Infosys](https://www.infosys.com). Connect with us on social media platforms to stay updated on our latest innovations and initiatives.

---

**Become a part of the Infosys journey, where technology meets human potential. Together, we can shape the future!**